In [1]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1109601510),
             ('parentid', 1109601379),
             ('minor', ''),
             ('user', 'Eden5'),
             ('timestamp',
              time.struct_time(tm_year=2022, tm_mon=9, tm_mday=10, tm_hour=20, tm_min=45, tm_sec=38, tm_wday=5, tm_yday=253, tm_isdst=-1)),
             ('comment',
              'Undid revision 1109601379 by [[Special:Contributions/Staxringold|Staxringold]] ([[User talk:Staxringold|talk]])')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [8]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [11]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [10]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [11]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [12]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.750000
2009-08-05,1,0.748121,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-14,1,0.930022,0.000000
2009-10-13,2,-0.227499,0.500000
...,...,...,...
2022-08-29,2,-0.125797,0.500000
2022-09-01,1,-0.995530,1.000000
2022-09-02,3,-0.353947,0.666667
2022-09-06,1,0.809337,0.000000


In [13]:
edits_df.index = pd.to_datetime(edits_df.index)

In [14]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [15]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [16]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2022-09-04,0,0.000000,0.00
2022-09-05,0,0.000000,0.00
2022-09-06,1,0.809337,0.00
2022-09-07,0,0.000000,0.00


In [17]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [18]:
rolling_edits = rolling_edits.dropna()

In [19]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2022-09-04,0.400000,-0.108480,0.150000
2022-09-05,0.400000,-0.108480,0.150000
2022-09-06,0.433333,-0.081502,0.150000
2022-09-07,0.433333,-0.081502,0.150000


In [21]:
rolling_edits.to_csv("wikipedia_edits.csv")